In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import random
import nltk 
import re
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense,Embedding,LSTM,GRU,Bidirectional, BatchNormalization, Conv1D, GlobalMaxPooling1D, Concatenate
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
import string
import gensim
nltk.download('punkt')
from nltk.tokenize import word_tokenize

print("Tensorflow Version",tf.__version__)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Tensorflow Version 2.4.1


In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
#Connect to the TPU handle and initialise it
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
import pandas as pd
df=pd.read_csv('/content/gdrive/My Drive/sentiment.csv',encoding='latin-1',header=None)

In [5]:
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
df.columns = ['sentiment', 'id', 'date', 'query', 'user_id', 'text']

In [7]:
df.head()

,sentiment,id,date,query,user_id,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [8]:
df = df.drop(['id', 'date', 'query', 'user_id'], axis=1)
df.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [9]:
df.sentiment.replace({0:"Negtive",4:"Positive"},inplace=True)
df.head()

,sentiment,text
0,Negtive,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,Negtive,is upset that he can't update his Facebook by ...
2,Negtive,@Kenichan I dived many times for the ball. Man...
3,Negtive,my whole body feels itchy and like its on fire
4,Negtive,"@nationwideclass no, it's not behaving at all...."


In [10]:
stop_words=set(stopwords.words('english'))
stop_words.remove('not')

In [11]:
def preprocess(df):
  corpus=[]
  # stemmer = SnowballStemmer("english")

  lemmatizer = WordNetLemmatizer()

  for i in range(0,len(df)):
      review=re.sub('@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+',' ',df['text'][i])#delete special symbol
      review=review.lower() #changg to lowercase
      review=review.split()   
      # review=[stemmer.stem(word) for word in review if not word in stop_words] #delete stop words
      for word, tag in pos_tag(review):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        if word not in stop_words:
            review.append(lemmatizer.lemmatize(word, pos))
      # review=[lemmatizer.lemmatize(word, pos) for word in review if not word in stop_words]
      review=' '.join(review) #joins  into one string
      corpus.append(review)
  return corpus

In [12]:
df.text=preprocess(df)
df.head()

,sentiment,text
0,Negtive,awww that s a bummer you shoulda got david car...
1,Negtive,is upset that he can t update his facebook by ...
2,Negtive,i dived many times for the ball managed to sav...
3,Negtive,my whole body feels itchy and like its on fire...
4,Negtive,no it s not behaving at all i m mad why am i h...


In [13]:
target_values = {'Negtive': 0 , 'Positive' : 1}
target = df['sentiment']
df.drop('sentiment', axis=1, inplace=True)
target = target.map(target_values)

In [14]:
df = pd.concat([target,df], axis=1)
df.head()

,sentiment,text
0,0,awww that s a bummer you shoulda got david car...
1,0,is upset that he can t update his facebook by ...
2,0,i dived many times for the ball managed to sav...
3,0,my whole body feels itchy and like its on fire...
4,0,no it s not behaving at all i m mad why am i h...


In [15]:
sentences = df.text
labels = df.sentiment

In [16]:
embedding_dim = 100
max_length = 16
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size=1280000

In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
vocab_size=len(word_index)
sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [19]:
train_padded, test_padded, train_labels, test_labels = train_test_split(padded, labels, test_size=0.33, random_state=42)

In [20]:
train_padded = np.array(train_padded)
train_labels = np.array(train_labels)
test_padded = np.array(test_padded)
test_labels = np.array(test_labels)

In [21]:
train_padded

array([[ 60287,     35,    619, ...,     53,  60287,    619],
       [   487,      1,    366, ...,      0,      0,      0],
       [   194,   1054,    171, ...,      0,      0,      0],
       ...,
       [     1,    756,   4110, ...,      0,      0,      0],
       [ 11433,     53, 196666, ...,   1715,    235,   1846],
       [    63,   1037,     14, ...,    415,    846,   9255]], dtype=int32)

In [22]:
embeddings_index = {};
with open('/content/gdrive/My Drive//glove.6B.100d.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

embeddings_matrix1 = np.zeros((vocab_size+1, embedding_dim));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix1[i] = embedding_vector;

In [23]:
tweets_list=list()
lines=df['text'].values.tolist()

for line in lines:
  tokens=word_tokenize(line)
  tweets_list.append(tokens)

In [24]:
df.head()

,sentiment,text
0,0,awww that s a bummer you shoulda got david car...
1,0,is upset that he can t update his facebook by ...
2,0,i dived many times for the ball managed to sav...
3,0,my whole body feels itchy and like its on fire...
4,0,no it s not behaving at all i m mad why am i h...


In [25]:
print(len(tweets_list))

1600000


In [26]:
import gensim
model=gensim.models.Word2Vec(sentences=tweets_list,size=100,window=5,workers=4,min_count=1)
words=list(model.wv.vocab)
print(len(words))

337023


In [27]:
model.wv.most_similar('man')

[('boy', 0.6790554523468018),
 ('dude', 0.6582843065261841),
 ('girl', 0.6058543920516968),
 ('woman', 0.6044704914093018),
 ('guy', 0.5997777581214905),
 ('shit', 0.5797611474990845),
 ('fella', 0.5472879409790039),
 ('yeah', 0.5453099012374878),
 ('lol', 0.5414555072784424),
 ('kid', 0.535315752029419)]

In [28]:
filename='tweet_embedding_word2vec.txt'
model.wv.save_word2vec_format(filename,binary=False)

In [29]:
embeddings_index = {};
with open('tweet_embedding_word2vec.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

embeddings_matrix2 = np.zeros((vocab_size+1, embedding_dim));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix2[i] = embedding_vector;

In [30]:
with strategy.scope():
  model1= Sequential([
        Embedding(vocab_size+1, embedding_dim, input_length=max_length, 
                                weights=[embeddings_matrix1], trainable=False),
        Conv1D(128, 3, activation='relu'),
        BatchNormalization(),
        Conv1D(64, 3, activation='relu'),
        GlobalMaxPooling1D(),
        BatchNormalization()
    ])
  
  model2=Sequential([
        Embedding(vocab_size+1, embedding_dim, input_length=max_length, 
                  weights=[embeddings_matrix2], trainable=False),
        Bidirectional(LSTM(units=32, return_sequences=True)),
        Bidirectional(LSTM(units=16)),
        BatchNormalization()
  ])

  modelOut = Concatenate()([model1.output, model2.output])
  modelOut = Dense(64, activation='relu')(modelOut)
  modelOut = Dense(32, activation='relu')(modelOut)
  modelOut = Dense(1, activation='sigmoid')(modelOut)

  model = Model([model1.input,model2.input], modelOut)

  model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])

In [31]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
embedding_input (InputLayer)    [(None, 16)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 16, 100)      33702900    embedding_input[0][0]            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 14, 128)      38528       embedding[0][0]                  
__________________________________________________________________________________________________
embedding_1_input (InputLayer)  [(None, 16)]         0                                            
______________________________________________________________________________________________

In [32]:
from keras.callbacks import ModelCheckpoint
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5),
              ModelCheckpoint(filepath='weights_best.hdf5',
                              save_weights_only=True,
                              monitor='val_accuracy',
                              mode='max',
                              save_best_only=True)]

In [ ]:
num_epochs = 10
history = model.fit([train_padded, train_padded],
                    train_labels, 
                    batch_size=16*strategy.num_replicas_in_sync,
                    epochs=num_epochs, 
                    validation_data=([test_padded, test_padded], test_labels),
                    verbose = 1, 
                    callbacks=callbacks)

print("Training Complete")

Epoch 1/10
 789/8375 [=>............................] - ETA: 1:53 - loss: 0.3656 - accuracy: 0.8351

In [198]:
# Loading best model

# model.load_weights("/content/weights_best.hdf5")

In [34]:
score = model.evaluate([test_padded, test_padded], test_labels, batch_size=16*strategy.num_replicas_in_sync)
print()
print("ACCURACY:",score[1])
print("LOSS:",score[0])

4125/4125 [==============================] - 51s 12ms/step - loss: 0.4145 - accuracy: 0.8103

ACCURACY: 0.810274600982666
LOSS: 0.4144524037837982


In [35]:
def testPreprocess(text, tokenizer):
  max_length = 16
  trunc_type='post'
  padding_type='post'

  # stemmer = SnowballStemmer("english")
  lemmatizer = WordNetLemmatizer()

  review=re.sub('@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+',' ',text)#delete special symbol
  review=review.lower() #changg to lowercase
  review=review.split()   
  # review=[stemmer.stem(word) for word in review if not word in stop_words] #delete stop words
  for word, tag in pos_tag(review):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        if word not in stop_words:
            review.append(lemmatizer.lemmatize(word, pos))
  # review=[lemmatizer.lemmatize(word, pos) for word in review if not word in stop_words]
  sentence =' '.join(review) #joins  into one string
  sent = []
  sent.append(sentence)
  sequences = tokenizer.texts_to_sequences(sent)
  padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  padded = np.array(padded)
  print(padded)
  return padded

In [36]:
def pred(text, threshold):
  value = model.predict([text, text])[0]
  print(value)
  if(value < threshold):
    return "Negative"
  else:
    return "Positive"

In [38]:
text = input()
text = testPreprocess(text, tokenizer)
threshold = 0.4
print(pred(text, threshold))


this a bad product
[[  49    4   87 2516   87 2516    0    0    0    0    0    0    0    0
     0    0]]
[0.0791817]
Negative
